In [18]:
import pandas as pd
import matplotlib.pyplot as plt

In [19]:
actual_data = pd.read_excel('input/Данные с 2022-wk40 по 2024-wk4.xlsx', skiprows=[0])
actual_data = actual_data.rename(columns = {'Unnamed: 0': 'date', 'ОРВИ': 'ari', 
                                            'Население': 'population', 
                                            'Число образцов тестированных на грипп': 'tested',
                                            'A (субтип не определен)': 'A_non_defined',
                                            })
actual_data.columns

Index(['date', 'ari', 'population', 'Unnamed: 3', 'Unnamed: 4', 'tested',
       'A_non_defined', 'A(H1)pdm09', 'A(H3)', 'B',
       'Число образцов тестированных на другие ОРВИ', 'Парагрипп',
       'Аденовирус', 'РС-вирус', 'Риновирус', 'Коронавирус', 'Метапневмовирус',
       'Бокавирус', 'Число образцов тестированных на COVID-19', 'COVID-19'],
      dtype='object')

In [23]:
actual_data_preprocessed = actual_data[['date', 'ari', 'population', 'tested', 
                                        'A_non_defined', 'A(H1)pdm09', 'A(H3)', 'B']]
actual_data_preprocessed['date'] = actual_data_preprocessed['date'].str.replace(u'\xa0', u' ')
dates = list(actual_data_preprocessed['date'])
dates = [elem.replace(u'\xa0', u' ') for elem in dates]
dates = [elem.split('(')[0] for elem in dates]
years = [int(elem.split('.')[0]) for elem in dates]
weeks = [int(elem.split('.')[1]) for elem in dates]
actual_data_preprocessed.insert(loc=0, column='num_of_week', value=weeks)
actual_data_preprocessed.insert(loc=0, column='year', value=years)
actual_data_preprocessed['date']
actual_data_final = actual_data_preprocessed.drop(columns=['date'])


def divide_A_non_defined_to_H1(row):
    if row['A(H1)pdm09'] + row['A(H3)'] == 0:
        return 0
    else:
        val = row['A(H1)pdm09'] + row['A_non_defined']*\
        row['A(H1)pdm09']/(row['A(H1)pdm09'] + row['A(H3)'])
        return val
    
    
def divide_A_non_defined_to_H3(row):
    if row['A(H1)pdm09'] + row['A(H3)'] == 0:
        return 0
    else:
        val = row['A(H3)'] + row['A_non_defined']*\
        row['A(H3)']/(row['A(H1)pdm09'] + row['A(H3)'])
        return val
    
actual_data_final['A(H1)pdm09_abs'] = actual_data_final.apply(divide_A_non_defined_to_H1, axis=1)
actual_data_final['A(H3)_abs'] = actual_data_final.apply(divide_A_non_defined_to_H3, axis=1)
actual_data_final['B_abs'] = actual_data_final['B']
actual_data_final['A(H1)pdm09_rel'] = actual_data_final['A(H1)pdm09_abs']/actual_data_final['tested']
actual_data_final['A(H3)_rel'] = actual_data_final['A(H3)_abs']/actual_data_final['tested']
actual_data_final['B_rel'] = actual_data_final['B_abs']/actual_data_final['tested']
actual_data_final['A(H1)pdm09_total'] = (actual_data_final['A(H1)pdm09_rel']*actual_data_final['ari']).astype(int)
actual_data_final['A(H3)_total'] = (actual_data_final['A(H3)_rel']*actual_data_final['ari']).astype(int)
actual_data_final['B_total'] = (actual_data_final['B_rel']*actual_data_final['ari']).astype(int)



year = 2023
actual_data_final = actual_data_final[actual_data_final['year'] == year]
real_input = pd.read_csv('input/incidence_strains_spb_2_age_groups.csv')
real_input = real_input[real_input['Год'] != year]

add_df = pd.DataFrame(columns=['index', 'Год', 'Неделя', 'A(H1N1)pdm09_15 и ст.', 
                               'A(H3N2)_15 и ст.',
       'B_15 и ст.', 'Население 15 и ст.', 'A(H1N1)pdm09_15 и ст._rel',
       'A(H3N2)_15 и ст._rel', 'B_15 и ст._rel', 'A(H1N1)pdm09_0-14',
       'A(H1N1)pdm09_0-14_rel', 'Население 0-14', 'A(H3N2)_0-14',
       'A(H3N2)_0-14_rel', 'B_0-14', 'B_0-14_rel'])

add_df['Год'] = actual_data_final['year'] 
add_df['Население 15 и ст.'] = actual_data_final['population']
add_df['Неделя'] = actual_data_final['num_of_week']
add_df['A(H1N1)pdm09_15 и ст.'] = actual_data_final['A(H1)pdm09_total']
add_df['A(H3N2)_15 и ст.'] = actual_data_final['A(H3)_total']
add_df['B_15 и ст.'] = actual_data_final['B_total']
add_df['A(H1N1)pdm09_15 и ст._rel'] = actual_data_final['A(H1)pdm09_rel']
add_df['A(H3N2)_15 и ст._rel'] = actual_data_final['A(H3)_rel']
add_df['B_15 и ст._rel'] = actual_data_final['B_rel']
add_df = add_df.fillna(0)
add_df

real_input = pd.concat([real_input, add_df], ignore_index=True)
real_input['index'] = real_input.index
real_input = real_input.sort_values(by=['Год', 'Неделя'])
real_input.to_csv('input/incidence_strains_spb_2_age_groups.csv', index=False)
real_input
# actual_data_final[actual_data_final['year'] == year]

/tmp/ipykernel_333297/1760189163.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual_data_preprocessed['date'] = actual_data_preprocessed['date'].str.replace(u'\xa0', u' ')
/tmp/ipykernel_333297/1760189163.py:65: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  add_df = add_df.fillna(0)


,index,Год,Неделя,A(H1N1)pdm09_15 и ст.,A(H3N2)_15 и ст.,B_15 и ст.,Население 15 и ст.,A(H1N1)pdm09_15 и ст._rel,A(H3N2)_15 и ст._rel,B_15 и ст._rel,A(H1N1)pdm09_0-14,A(H1N1)pdm09_0-14_rel,Население 0-14,A(H3N2)_0-14,A(H3N2)_0-14_rel,B_0-14,B_0-14_rel
0,0,2010.0,1.0,NaN,NaN,NaN,4042163.0,NaN,NaN,NaN,NaN,NaN,539691.0,NaN,NaN,NaN,NaN
1,1,2010.0,2.0,4084.0,0.0,0.0,4042163.0,1.010350,0.000000,0.000000,4037.0,25.352802,539691.0,0.0,0.0,0.0,0.0
2,2,2010.0,3.0,2189.0,0.0,0.0,4042163.0,0.541542,0.000000,0.000000,2464.0,15.083921,539691.0,0.0,0.0,0.0,0.0
3,3,2010.0,4.0,5412.0,0.0,0.0,4042163.0,1.338887,0.000000,0.000000,6455.0,38.539335,539691.0,0.0,0.0,0.0,0.0
4,4,2010.0,5.0,NaN,NaN,NaN,4042163.0,NaN,NaN,NaN,NaN,NaN,539691.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,645,2023.0,52.0,3805.0,109096.0,223.0,54097055.0,0.006732,0.192983,0.000396,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
590,590,2024.0,1.0,0.0,56535.0,284.0,54070728.0,0.000000,0.217018,0.001093,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
591,591,2024.0,2.0,68.0,89705.0,530.0,54437820.0,0.000147,0.193533,0.001145,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
592,592,2024.0,3.0,75.0,79389.0,459.0,54743508.0,0.000172,0.180114,0.001042,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
